In [49]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Simple Moving Average function
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

# SMA Cross Strategy class
class SmaCross(Strategy):
    n1 = 30
    n2 = 100

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()



In [50]:
import yfinance as yf

# Get BTC-USD data from yfinance
data = yf.download('BTC-USD')

# Code for running the backtest.
# Assumes the existence of `data` variable containing backtest data.
bt = Backtest(data, SmaCross)
stats = bt.run()


[*********************100%%**********************]  1 of 1 completed


C:\Users\Chinook\AppData\Local\Temp\ipykernel_14772\2896767434.py:8: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SmaCross)


In [51]:
#print(stats['_trades'].to_string())
stats


Start                     2014-09-17 00:00:00
End                       2023-10-21 00:00:00
Duration                   3321 days 00:00:00
Exposure Time [%]                   94.400963
Equity Final [$]                319831.739594
Equity Peak [$]                  617868.64975
Return [%]                        3098.317396
Buy & Hold Return [%]             6461.795808
Return (Ann.) [%]                   46.335975
Volatility (Ann.) [%]               129.67155
Sharpe Ratio                         0.357333
Sortino Ratio                        0.929295
Calmar Ratio                         0.491756
Max. Drawdown [%]                  -94.225502
Avg. Drawdown [%]                  -14.754125
Max. Drawdown Duration      727 days 00:00:00
Avg. Drawdown Duration       64 days 00:00:00
# Trades                                   28
Win Rate [%]                        46.428571
Best Trade [%]                    1113.534101
Worst Trade [%]                    -64.725432
Avg. Trade [%]                    

In [41]:
bt.plot()

f:\Python\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
f:\Python\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1648', ...)

In [39]:
# Define a range of values to test for each parameter
param_grid = {'n1': range(5, 60, 5), 'n2': range(10, 90, 5)}
# Run the optimization
res = bt.optimize(**param_grid)

# Print the best results and the parameters that lead to these results
print("Best result: ", res['Return [%]'])
print("Parameters for best result: ", res['_strategy'])

Best result:  30052.376494750977
Parameters for best result:  SmaCross(n1=10,n2=45)


In [35]:
res

Start                     2014-09-17 00:00:00
End                       2023-10-21 00:00:00
Duration                   3321 days 00:00:00
Exposure Time [%]                   98.254064
Equity Final [$]               3015237.649475
Equity Peak [$]                4007171.876038
Return [%]                       30052.376495
Buy & Hold Return [%]              6464.85532
Return (Ann.) [%]                   87.245724
Volatility (Ann.) [%]               129.49466
Sharpe Ratio                          0.67374
Sortino Ratio                         2.14152
Calmar Ratio                         1.363344
Max. Drawdown [%]                  -63.993916
Avg. Drawdown [%]                  -10.885191
Max. Drawdown Duration      637 days 00:00:00
Avg. Drawdown Duration       36 days 00:00:00
# Trades                                   79
Win Rate [%]                        46.835443
Best Trade [%]                     348.914788
Worst Trade [%]                    -20.891945
Avg. Trade [%]                    

In [45]:
res['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
2014-09-17,1.000000e+04,0.000000,NaT
2014-09-18,1.000000e+04,0.000000,NaT
2014-09-19,1.000000e+04,0.000000,NaT
2014-09-20,1.000000e+04,0.000000,NaT
2014-09-21,1.000000e+04,0.000000,NaT
...,...,...,...
2023-10-17,2.887846e+06,0.279331,NaT
2023-10-18,2.879018e+06,0.281534,NaT
2023-10-19,2.918556e+06,0.271667,NaT
2023-10-20,3.015833e+06,0.247391,NaT
